# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

I choose TikTok wikipedia page for dataset. I aimed to investigate recent trends in TikTok and recent informations about the company. Since GPT 3.5 is not trained on 2023 and after data, it can not be aware of recent trends.

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [1]:
import openai
openai.api_key = "sk-proj-MvKLgSSrzXCS9G036IL0T3BlbkFJwFnqrQGL39QBpVqKOGEu"

In [2]:
from dateutil.parser import parse
import pandas as pd
import requests

# Get the Wikipedia page for "TikTok" for recent updates & trends
resp = requests.get("https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exlimit=1&titles=TikTok&explaintext=1&formatversion=2&format=json")

# Load page text into a dataframe
df = pd.DataFrame()
df["text"] = resp.json()["query"]["pages"][0]["extract"].split("\n")

print(len(df["text"]))
df["text"].head(30)

361


0     TikTok, whose mainland Chinese counterpart is ...
1     Since its launch, TikTok has become one of the...
2     TikTok has come under scrutiny due to data pri...
3                                                      
4                                                      
5                             == Corporate structure ==
6     TikTok Ltd was incorporated in the Cayman Isla...
7     Its parent company, Beijing-based ByteDance, i...
8                                                      
9                                                      
10                                        == History ==
11                                                     
12                                                     
13                                       === Douyin ===
14                                                     
15    Douyin was launched by ByteDance in September ...
16    While TikTok and Douyin share a similar user i...
17                                              

In [3]:
# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]
df

,text
0,"TikTok, whose mainland Chinese counterpart is ..."
1,"Since its launch, TikTok has become one of the..."
2,TikTok has come under scrutiny due to data pri...
6,TikTok Ltd was incorporated in the Cayman Isla...
7,"Its parent company, Beijing-based ByteDance, i..."
...,...
346,Timeline of social media
357,Media related to TikTok at Wikimedia Commons
358,Works on the topic TikTok at Wikisource
359,Official website (in English)


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [4]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

response = openai.Embedding.create(
    input=df["text"].tolist(),
    engine=EMBEDDING_MODEL_NAME
)

# Extract embeddings
embeddings = [data["embedding"] for data in response["data"]]

In [5]:
df["embeddings"] = embeddings
df

,text,embeddings
0,"TikTok, whose mainland Chinese counterpart is ...","[-0.023847214877605438, -0.009270330891013145,..."
1,"Since its launch, TikTok has become one of the...","[-0.031663309782743454, -0.015538003295660019,..."
2,TikTok has come under scrutiny due to data pri...,"[-0.016898926347494125, -0.016976090148091316,..."
6,TikTok Ltd was incorporated in the Cayman Isla...,"[-0.004194025881588459, -0.015445277094841003,..."
7,"Its parent company, Beijing-based ByteDance, i...","[0.016316397115588188, -0.026677049696445465, ..."
...,...,...
346,Timeline of social media,"[-0.04013114795088768, -0.00789528340101242, -..."
357,Media related to TikTok at Wikimedia Commons,"[-0.012876149266958237, -0.0044457376934587955..."
358,Works on the topic TikTok at Wikisource,"[-0.01904177851974964, -0.009272518567740917, ..."
359,Official website (in English),"[-0.0020897709764540195, -0.006642256397753954..."


In [6]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings


def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [7]:
question1 = "What is the most popular trend of 2023?"
question2 = "What is the revenue of TikTok in 2023?"

In [8]:
resp1 = get_rows_sorted_by_relevance(question1, df)
resp1

,text,embeddings,distances
75,"In 2023, a trend emerged where streamers acted...","[-0.021095650270581245, -0.04050572216510773, ...",0.182548
1,"Since its launch, TikTok has become one of the...","[-0.031663309782743454, -0.015538003295660019,...",0.191429
88,Content promoting cosmetic surgery is popular ...,"[-0.035577356815338135, 0.01277468167245388, 0...",0.206899
344,Cultural impact of TikTok,"[-0.02797224186360836, -0.02191379852592945, 0...",0.213185
345,List of most-followed TikTok accounts,"[-0.025071294978260994, -0.010483878664672375,...",0.213460
...,...,...,...
315,According to a 2020 article in The New York Ti...,"[-0.010271196253597736, -0.007845335640013218,...",0.296114
55,The app allows users to set their accounts as ...,"[-0.022631125524640083, -0.016209479421377182,...",0.296770
359,Official website (in English),"[-0.0020897709764540195, -0.006642256397753954...",0.301191
73,"On 10 August 2020, Emily Jacobssen wrote and s...","[-0.013269932009279728, -0.008008637465536594,...",0.304753


In [9]:
resp1['text'][75]

'In 2023, a trend emerged where streamers acted as if they were video-game characters following prompts from their viewers.'

In [10]:
resp2 = get_rows_sorted_by_relevance(question2, df)
resp2

,text,embeddings,distances
27,According to data from app analytics group Sen...,"[-0.008965987712144852, -0.030806545168161392,...",0.099590
36,"In 2023, several high-level executives transfe...","[-0.01912267878651619, -0.04586224630475044, 0...",0.123755
60,"In May 2022, TikTok announced TikTok Pulse, an...","[-0.0274394191801548, -0.024557892233133316, -...",0.147734
25,"In May 2021, TikTok appointed Shou Zi Chew as ...","[-0.011664737947285175, -0.026164883747696877,...",0.151094
40,Although the size of its user base falls short...,"[-0.030518608167767525, -0.015180171467363834,...",0.156338
...,...,...,...
287,The Wall Street Journal has reported that doct...,"[-0.024646127596497536, 0.020181341096758842, ...",0.270750
91,"In December 2022, it was reported that a cosme...","[-0.04501093924045563, 0.013289311900734901, 0...",0.276171
359,Official website (in English),"[-0.0020897709764540195, -0.006642256397753954...",0.292316
297,Several former employees of the company have c...,"[0.0057318066246807575, -0.014018673449754715,...",0.303964


In [11]:
resp2['text'][27]

"According to data from app analytics group Sensor Tower, advertising on TikTok in the US grew by 11% in March 2023, with companies including Pepsi, DoorDash, Amazon and Apple among the top spenders. According to estimates from research group Insider Intelligence, TikTok is projected to generate $14.15 billion in revenue in 2023, up from $9.89 billion in 2022. In March 2024, The Wall Street Journal reported that TikTok's growth in the U.S. had stagnated."

In [12]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
        Answer the question based on the context below, and if the question
        can't be answered based on the context, say "I don't know"

        Context: 

        {}

        ---

        Question: {}
        Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

In [13]:
print(create_prompt(question1, df, 200))


        Answer the question based on the context below, and if the question
        can't be answered based on the context, say "I don't know"

        Context: 

        In 2023, a trend emerged where streamers acted as if they were video-game characters following prompts from their viewers.

###

Since its launch, TikTok has become one of the world's most popular social media platforms, using recommendation algorithms to connect content creators with new audiences. Many of its users are young and part of Generation Z. In April 2020, TikTok surpassed two billion mobile downloads worldwide. Cloudflare ranked TikTok the most popular website of 2021, surpassing Google. The popularity of TikTok has allowed viral trends in food and music to take off and increase the platform's cultural impact worldwide.

        ---

        Question: What is the most popular trend of 2023?
        Answer:


In [14]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""
        

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [15]:
prompt1 = """
Question: "What is the most popular trend of 2023?"
Answer:
"""
initial_answer1 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt1,
    max_tokens=150
)

initial_answer1 = initial_answer1["choices"][0]["text"].strip()
print(initial_answer1)

It is impossible to accurately predict the most popular trend of 2023 as trends are constantly evolving and can change quickly. It will largely depend on the societal, cultural, and technological developments in the coming years.


In [16]:
custom_answer1 = answer_question(question1, df)
print(custom_answer1)

Acting as video-game characters and following prompts from viewers as a streamer on TikTok.


### Question 2

In [17]:
prompt2 = """
Question: "What is the revenue of TikTok in 2023?"
Answer:
"""
initial_answer2 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt2,
    max_tokens=150
)

initial_answer2 = initial_answer2["choices"][0]["text"].strip()
print(initial_answer2)

It is not possible to accurately predict the revenue of TikTok in 2023 as it will depend on various factors such as user growth, advertising revenue, competition, and market trends. Additionally, TikTok has not released any projections for their revenue in 2023.


In [18]:
custom_answer2 = answer_question(question2, df)
print(custom_answer2)

$14.15 billion
